<a href="https://colab.research.google.com/github/hariprabha92/ML-and-NLP/blob/master/NLPWorkshop2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from  sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

In [0]:
data_yelp = pd.read_csv('yelp.txt', sep='\t', header = None)
data_yelp.head()


,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
columns_name = ['Review','sentiment']
data_yelp.columns = columns_name
data_yelp.head()

,Review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
data_amazon = pd.read_csv('amazon.txt', sep='\t', header = None)
data_amazon.columns = columns_name
data_amazon.head()


,Review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [0]:
data_imdb = pd.read_csv('imdb.txt', sep='\t', header = None)
data_imdb.columns = columns_name
data_imdb.head

<bound method NDFrame.head of                                                 Review  sentiment
0    A very, very, very slow-moving, aimless movie ...          0
1    Not sure who was more lost - the flat characte...          0
2    Attempting artiness with black & white and cle...          0
3         Very little music or anything to speak of.            0
4    The best scene in the movie was when Gerardo i...          1
..                                                 ...        ...
743  I just got bored watching Jessice Lange take h...          0
744  Unfortunately, any virtue in this film's produ...          0
745                   In a word, it is embarrassing.            0
746                               Exceptionally bad!            0
747  All in all its an insult to one's intelligence...          0

[748 rows x 2 columns]>

In [0]:
data = data_imdb.append(data_amazon, ignore_index=True)
data = data_imdb.append(data_yelp, ignore_index=True)
data.isnull().sum()


Review       0
sentiment    0
dtype: int64

In [0]:
import string
punct = string.punctuation
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
stopwords= list(STOP_WORDS)

In [0]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [31]:
text_data_cleaning("    Hello how are you. Like this video")


['hello', 'like', 'video']

In [0]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)


X = data['Review']
y = data['sentiment']

In [46]:
classifier = LinearSVC()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x7f6f1a22da60>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [0]:
y_pred = clf.predict(X_test)

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.75      0.76       167
           1       0.78      0.79      0.78       183

    accuracy                           0.77       350
   macro avg       0.77      0.77      0.77       350
weighted avg       0.77      0.77      0.77       350



In [50]:
clf.predict(["Wrong product"])

array([1])